# 24.1

Write a python program using SQLObject to lookup the scientific name for a user-supplied organism name.

In [1]:
from sqlobject import *
import os.path

generic_name = 'human'
taxa_id = 9606

In [2]:
db = 'small_taxa.db3'
conn_str = os.path.abspath(db)
conn = 'sqlite:' + conn_str

sqlhub.processConnection = connectionForURI(conn)

class Taxonomy(SQLObject):
    class sqlmeta:
        fromDatabase = True

class Name(SQLObject):
    class sqlmeta:
        fromDatabase = True

In [3]:
Name._connection.debug = True
name_schema = Name.select(
                OR(Name.q.name == f"{generic_name}",
                    LIKE(Name.q.name, f"%{generic_name}%")))
list(name_schema)

 1/Select  :  SELECT name.id, name.taxonomy_id, name.name, name.name_class FROM name WHERE (((name.name) = ('human')) OR (name.name LIKE ('%human%')))
 1/QueryR  :  SELECT name.id, name.taxonomy_id, name.name, name.name_class FROM name WHERE (((name.name) = ('human')) OR (name.name LIKE ('%human%')))
 1/COMMIT  :  auto


[<Name 377 taxonomyID=141 name='human' nameClass="'genbank common n...'">]

In [4]:
Taxonomy._connection.debug = True
taxa_schema = Taxonomy.select(Taxonomy.q.taxid==taxa_id)
list(taxa_schema)

 1/Select  :  SELECT taxonomy.id, taxonomy.taxid, taxonomy.scientific_name, taxonomy.rank, taxonomy.parent_id FROM taxonomy WHERE ((taxonomy.taxid) = (9606))
 1/QueryR  :  SELECT taxonomy.id, taxonomy.taxid, taxonomy.scientific_name, taxonomy.rank, taxonomy.parent_id FROM taxonomy WHERE ((taxonomy.taxid) = (9606))
 1/COMMIT  :  auto


[<Taxonomy 141 taxid=9606 scientificName='Homo sapiens' rank='species' parentID=140>]

In [ ]:
# find taxonomic id based on generic name(s)
name_search = Name.select(
                OR(Name.q.name == f"{generic_name}",
                    LIKE(Name.q.name, f"%{generic_name}%")))
gen_names = list(name_search)

results = []
for n in gen_names:
    print(type(n))
    taxa_id = n.q.taxonomy_id
    taxa_search = Taxonomy.select(Taxonomy.q.taxid==taxa_id)
    for result in taxa_search:
        print(result)
        results.append(result.scientific_name)